<a href="https://colab.research.google.com/github/yusuku/DepthEstimationFrom360/blob/main/SimplePytorchOnnxModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install onnxruntime
!pip install onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 30.6 MB/s eta 0:00:00


In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import onnxruntime as ort



In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model = NeuralNetwork().to(device)

Using cpu device


In [11]:
torch_input=torch.randn((1,1,28,28))
torch_output=model(torch_input)

In [16]:
torch_output

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0252, 0.0000, 0.1330, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)

In [5]:
torch_output.shape

torch.Size([1, 10])

In [10]:
model.eval()
torch.onnx.export(model,               # model being run
                  torch_input,                         # model input (or a tuple for multiple inputs)
                  "sup.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=15,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [14]:
ort_sess = ort.InferenceSession('/content/sup.onnx')
onnx_outputs = ort_sess.run(None, {'input': torch_input.numpy()})


In [15]:
onnx_outputs

[array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.02516446, 0.        , 0.13297555, 0.        , 0.        ]],
       dtype=float32)]